# Name of the Project

*By Daniel Deutsch, José Lucas Barretto, Kevin kuhl and Lucas Miguel Agrizzi*

In [1]:
import yfinance
import pandas as pd
from datetime import datetime

# Data Collection



In [ ]:
START_DATE = ""
END_DATE = ""

# Data Analysis